<a href="https://colab.research.google.com/github/Piyushlamsoge/Fraud_transcation/blob/main/Transaction_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Necessary package**

In [23]:
!pip install pandas faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Dataset Creation Using Faker**

In [20]:
# EXAMPLE 2

import pandas as pd
from faker import Faker
from random import randint, uniform, choice

# Set seed for reproducibility
Faker.seed(42)

# Create Faker instance
fake = Faker()

# Generate fake data
data = []
for _ in range(10000):
    transaction_id = fake.uuid4()
    user_id = randint(1, 1000)
    transaction_amount = round(uniform(1, 10000), 2)
    transaction_date = fake.date_between(start_date='-1y', end_date='today')
    is_foreign_transaction = randint(0, 1)
    transaction_type = choice(['purchase', 'refund', 'cash withdrawal'])
    data.append([transaction_id, user_id, transaction_amount, transaction_date, is_foreign_transaction, transaction_type])

# Create DataFrame from generated data
columns = ['transaction_id', 'user_id', 'transaction_amount', 'transaction_date', 'is_foreign_transaction', 'transaction_type']
df = pd.DataFrame(data, columns=columns)

# Save DataFrame to CSV
df.to_csv('fraud_dataset.csv', index=False)


# **Fraud Logic Code**

In [21]:
import pandas as pd
from datetime import datetime, timedelta

# Read the fraud dataset from the CSV file
df = pd.read_csv('fraud_dataset.csv')

# Define the fraud detection criteria
threshold_amount = 100000
threshold_locations = 3
threshold_days = 1

# Apply fraud detection logic
is_fraud = (
    (df['transaction_amount'] > threshold_amount) |
    (df.groupby('user_id')['transaction_type'].transform('nunique') > threshold_locations) |
    (pd.to_datetime(df['transaction_date']) >= datetime.now() - timedelta(days=threshold_days)) |
    (df['is_foreign_transaction'] == 1)
)

# Update the 'is_fraud' column in the dataframe
df['is_fraud'] = is_fraud.astype(int)

# Print flagged fraud transactions
fraud_transactions = df[df['is_fraud'] == 1]
print(fraud_transactions)


                            transaction_id  user_id  transaction_amount  \
0     bdd640fb-0667-4ad1-9c80-317fa3b1799d      896             8736.93   
1     bc8960a9-23b8-41e9-b924-56de3eb13b90      389             7469.28   
3     07a0ca6e-0822-48f3-ac03-1199972a8469      295             3633.25   
5     a65ed389-b74d-4fb1-b2e7-06298fadc1a6      189             2554.78   
7     01a9e71f-de8a-474b-8f36-d58b47378190      444             9752.11   
...                                    ...      ...                 ...   
9990  89199f95-678d-4bb9-9f9d-8ee5bd742407      205             4014.38   
9993  a3565dd0-ed88-4e88-9b18-4e8c7fccf867       36             6940.47   
9995  61a7442f-c772-42ba-9187-f45a9e47cedc      854             7991.35   
9998  02d8eb63-00fe-4ccc-bee9-d208161b0d82      565              227.53   
9999  a97c0644-239f-4c50-bafb-233e91968637       53             4373.96   

     transaction_date  is_foreign_transaction transaction_type  is_fraud  
0          2022-09-23   

# **Analysis of Fraud Detection system**

In [22]:
# Analyze the flagged fraud transactions
flagged_transactions = df[df['is_fraud'] == 1]

# Print the count of flagged fraud transactions
print("Total flagged fraud transactions:", len(flagged_transactions))

# Calculate the percentage of flagged fraud transactions
percentage_fraud_transactions = (len(flagged_transactions) / len(df)) * 100
print("Percentage of flagged fraud transactions: {:.2f}%".format(percentage_fraud_transactions))

# Print the distribution of flagged fraud transactions by transaction type
fraud_transactions_by_type = flagged_transactions['transaction_type'].value_counts()
print("Flagged fraud transactions distribution by transaction type:")
print(fraud_transactions_by_type)

# Calculate the average transaction amount of flagged fraud transactions
average_fraud_transaction_amount = flagged_transactions['transaction_amount'].mean()
print("Average transaction amount of flagged fraud transactions: {:.2f}".format(average_fraud_transaction_amount))

# Calculate the maximum transaction amount among flagged fraud transactions
max_fraud_transaction_amount = flagged_transactions['transaction_amount'].max()
print("Maximum transaction amount among flagged fraud transactions: {:.2f}".format(max_fraud_transaction_amount))

# Print the flagged fraud transactions with the highest transaction amount
highest_amount_fraud_transactions = flagged_transactions[flagged_transactions['transaction_amount'] == max_fraud_transaction_amount]
print("Flagged fraud transactions with the highest transaction amount:")
print(highest_amount_fraud_transactions)


Total flagged fraud transactions: 4952
Percentage of flagged fraud transactions: 49.52%
Flagged fraud transactions distribution by transaction type:
cash withdrawal    1697
purchase           1654
refund             1601
Name: transaction_type, dtype: int64
Average transaction amount of flagged fraud transactions: 4942.22
Maximum transaction amount among flagged fraud transactions: 9998.92
Flagged fraud transactions with the highest transaction amount:
                            transaction_id  user_id  transaction_amount  \
3769  33f92a87-8372-4123-9ed7-a652de2bfe58      566             9998.92   

     transaction_date  is_foreign_transaction transaction_type  is_fraud  
3769       2023-04-07                       1  cash withdrawal         1  
